In [1]:
import numpy as np
import pandas as pd
import scipy.stats as st
from scipy.optimize import curve_fit
import matplotlib.pyplot as plt
np.random.seed(42)

df = pd.DataFrame({
    'value': [ 
        1.26,   2.78,   0.63,   0.34,
        3.53,   4.10,   1.31,   0.91,
        11.98,  13.14,  9.86,   6.53,
        90.82,  97.12,  75.85,  59.46,
        83.45,  116.62, 104.00, 80.85,
        55.98,  67.28,  79.33,  82.66,
        66.32,  78.53,  69.10,  67.27,
        37.42,  55.35,  60.76,  73.20
    ], 
    'lambda': [ 
        2,    2,    2,    2,   
        7,    7,    7,    7,    
        12,   12,   12,   12,   
        19.5, 19.5, 19.5, 19.5, 
        29.5, 29.5, 29.5, 29.5, 
        39.5, 39.5, 39.5, 39.5, 
        54.5, 54.5, 54.5, 54.5, 
        75,   75,   75,   75    
    ]
})

xs = [float(i) for i in df['lambda'].unique()]
ys = [np.mean(df[ df['lambda'].values == x_i ].value.values) for x_i in xs]
data = [df[ df['lambda'].values == x_i ].value.values for x_i in xs]

In [17]:
def isiterable(p_object):
    try:
        it = iter(p_object)
    except TypeError: 
        return False
    return True

def eval(func, vars):
    if isiterable(vars):
        return [ func(var) for var in vars ]
    return func(vars)

def derivate(func, vars, h = 0.001):    
    f = lambda var: (func(var + h) - func(var - h))/(2*h)
    return eval(f, vars)

def eta(x, theta=[153.79, 17.26, -2.58, 0.455, 0.01746]): 
    f = lambda var: (theta[0] + theta[2]*var + theta[4]*var**2) * np.exp(theta[3]*(var-theta[1]))/( 1 + np.exp(theta[3]*(var-theta[1])) )
    return eval(f, x)

def y(x, theta=[153.79, 17.26, -2.58, 0.455, 0.01746], sigma=10.17): 
    f = lambda var: eta(var, theta) + np.random.normal(0, sigma**2)
    return eval(f, x)

def f(i, y, theta=[153.79, 17.26, -2.58, 0.455, 0.01746], sigma=10.17, x=xs):
    f = lambda var: np.exp( -(var - eta(x[i], theta))**2 / (2*sigma**2) ) / np.sqrt(2*np.pi*sigma**2)
    return eval(f, y)

def Lik(theta, y=ys):
    return np.prod( [f(i, y[i], theta) for i in range(len(y))] )

def L(theta, y=ys):
    return np.log( Lik(theta, y) )

def L_sum(theta, y=ys):
    return np.sum( [np.log(f(i, y[i], theta)) for i in range(len(y))] ) 

def derivate_L(theta, y=ys): 
    score = []
    for i in range(len(theta)):
        f = lambda var: L((theta[0:i]+[var]+theta[i+1:]), y)
        score.append( derivate(f, theta[i]) )
    return score

def hessian_L(theta, y=ys): return []

test = [153.79, 17.26, -2.58, 0.455, 0.01746]
for i in range(len(test)):
    tf = lambda var: L(test[0:i]+[var]+test[i+1:])
    print( derivate(tf, test[i]) )

-0.007170526544086897
0.0017119846162927388
-0.4880757704803784
-0.017327848876647067
-34.7611597401265


In [ ]:
# Likelyhood functions
# a=10.17, b=153.79, c=17.26, d=-2.58, e=0.455, f=0.01746 
print(Lik([153.79, 17.26, -2.58, 0.455, 0.01746]))
print(L([153.79, 17.26, -2.58, 0.455, 0.01746]))
print(L_sum([153.79, 17.26, -2.58, 0.455, 0.01746]))

In [ ]:
# Random MLE
max = 0
param = []
for i in range(100000):
    a = np.random.uniform(150, 160)
    b = np.random.uniform(15, 20)
    c = np.random.uniform(-5, 0)
    d = np.random.uniform(-2, 2)
    e = np.random.uniform(-2, 2)
    likelyHood = Lik([a, b, c, d, e], ys)
    if likelyHood>max: 
        max=likelyHood
        param = [a,b,c,d,e]
print(max)
print(param)

In [ ]:
# Data
plt.plot(xs, data, 'o', color='black', markersize='3')

# Regression
x = np.linspace(1, 80, num=100)
plt.plot(x, eta(x))

# TODO Confidence Bands
# plt.plot(x, [eta(x_i) - h_lower(x_i) for x_i in x], linestyle='dashed')
# plt.plot(x, [eta(x_i) + h_upper(x_i) for x_i in x], linestyle='dashed')